In [1]:
import pandas as pd

## dictionary of codes to formats (such as book and DVD)

In [3]:
dictionary = pd.read_csv('./seattle-library-checkout-records/Integrated_Library_System__ILS__Data_Dictionary.csv')
print(len(dictionary))
dictionary.head()

555


,Code,Description,Code Type,Format Group,Format Subgroup,Category Group,Category Subgroup
0,pkbknh,Peak Picks Book,ItemType,NaN,NaN,NaN,NaN
1,acart,Framed Art: Adult/YA,ItemType,Media,Art,NaN,NaN
2,acbk,Book: Adult/YA,ItemType,Print,Book,NaN,NaN
3,accas,Audio Tape: Adult/YA,ItemType,Media,Audio Tape,NaN,NaN
4,accd,CD: Adult/YA,ItemType,Media,Audio Disc,NaN,NaN


## inventory: BibNumber to Title, Author and other details of the item

### i simplified the inventory file a little, since it had a row for every set of copies of the same book

In [6]:
inventory = pd.read_csv('./seattle-library-checkout-records/Library_Collection_Inventory_2018.csv')
del inventory['ItemCount']
print(len(inventory))
inventory = inventory.drop_duplicates(subset=['BibNum','Title','Author','PublicationYear','Subjects','ItemType'])
print(len(inventory))

21506883
975637


In [4]:
inventory = pd.read_csv('./seattle-library-checkout-records/Library_Collection_Inventory_2018.csv')
del inventory['ItemCount']
print(len(inventory))
inventory = inventory.drop_duplicates(subset=['BibNum','Title','Author','PublicationYear','Subjects','ItemType'])
print(len(inventory))

2687149
591021


### I merge the inventory and dictionary on Code, to create one file with all the metadata

In [7]:
inventory = pd.merge(inventory.rename(columns = {'ItemType': 'Code'}), dictionary, on = 'Code', how='inner')

In [8]:
inventory.to_csv('./seattle-library-checkout-records/inventory_unique.csv', index=False)

In [2]:
inventory = pd.read_csv('./seattle-library-checkout-records/inventory_unique.csv')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## I groupby each yearly dataset by the BibNumber and join to the metadata table

This gives me a list of all the unique items that were checked out each year, alongside the number of check outs they had and the item's metadata

I then concat all the yearly unique check outs into one. The unified file has a manageable size because it contains only unique checkout items.

In [9]:
inventory = pd.read_csv('./seattle-library-checkout-records/inventory_unique.csv')
inventory.rename(columns = {'BibNum': 'BibNumber'}, inplace = True)

most_checkout = pd.DataFrame(columns = ['year' ,'num_checkouts', 'title', 'author', 'publication_year','publisher', 'format_subgroup','description'])

for year in range(2006, 2019):
    print(year)
    df = pd.read_csv('./seattle-library-checkout-records/Checkouts_By_Title_Data_Lens_'+str(year)+'.csv')
    most_checked_out = pd.merge(df.groupby('BibNumber').size().to_frame().rename(columns = {0: 'num_checkouts'}).sort_values(by='num_checkouts',ascending=False), inventory, on ='BibNumber', how='inner')
    most_checked_out.rename(columns = {'Title':'title',\
                                    'Author':'author',\
                                    'PublicationYear' : 'publication_year',\
                                    'Publisher' : 'publisher',\
                                   'Format Subgroup':'format_subgroup',\
                                   'Description' : 'description'}, inplace = True)
    
    most_checked_out['year'] = [year]*len(most_checked_out)
    
    most_checkout = pd.concat([most_checkout, most_checked_out[['year', 'title','author','publication_year','publisher','format_subgroup','description','num_checkouts']]])

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2006


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [10]:
print(len(most_checkout))
print(most_checkout.head())
most_checkout.to_csv('./seattle-library-checkout-records/most_checked_out_all.csv')

4892594
  author    description format_subgroup num_checkouts publication_year  \
0    NaN  DVD: Adult/YA      Video Disc          2564           [2006]   
1    NaN  DVD: Adult/YA      Video Disc          2564           [2006]   
2    NaN  DVD: Adult/YA      Video Disc          2341           [2006]   
3    NaN  DVD: Adult/YA      Video Disc          2341           [2006]   
4    NaN  DVD: Adult/YA      Video Disc          2316           [2005]   

                                           publisher  \
0  Touchstone Home Entertainment : Distributed by...   
1  Touchstone Home Entertainment : Distributed by...   
2                 Distributed by Universal Pictures,   
3                 Distributed by Universal Pictures,   
4                                 Warner Home Video,   

                                               title  year  
0  Flightplan / Touchstone Pictures and Imagine E...  2006  
1  Flightplan [videorecording] / Touchstone Pictu...  2006  
2  Broken flowers / Bac Fil

## create one dataframe with 10% sample size from each year

In order to do analysis on checkout trends (such as day of the week) we still need the dataset with all the checkouts.

I created one unified dataset from all the years by taking a random 10% sample of each year.

The sizes of each year are large enough that the 10% sample still perceives a lot of information.

In [11]:
inventory = pd.read_csv('./seattle-library-checkout-records/inventory_unique.csv')
inventory.rename(columns = {'BibNum': 'BibNumber'}, inplace = True)
checkouts = pd.DataFrame(columns = ['year', 'checkout_time', 'title', 'author', 'publication_year','publisher', 'format_subgroup','description'])

for year in range(2006, 2019):
    print(year)
    df = pd.read_csv('./seattle-library-checkout-records/Checkouts_By_Title_Data_Lens_'+str(year)+'.csv')
    df = pd.merge(df.sample(frac=0.1), inventory, on ='BibNumber', how='inner') ##take 10% sample, and join to metadata table
    df.rename(columns = {'Title':'title',\
                         'CheckoutDateTime': 'checkout_time',\
                        'Author':'author',\
                        'PublicationYear' : 'publication_year',\
                        'Publisher' : 'publisher',\
                       'Format Subgroup':'format_subgroup',\
                       'Description' : 'description'}, inplace = True)
    
    df['year'] = [year]*len(df)
    
    checkouts = pd.concat([checkouts, df[['year','checkout_time', 'title','author','publication_year','publisher','format_subgroup','description']]])

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [12]:
print(len(checkouts))
print(checkouts.head())

11926141
   year           checkout_time                                title  \
0  2006  12/30/2006 03:03:00 PM                    Document / R.E.M.   
1  2006  12/30/2006 03:03:00 PM  Document [sound recording] / R.E.M.   
2  2006  01/27/2006 05:03:00 PM                    Document / R.E.M.   
3  2006  01/27/2006 05:03:00 PM  Document [sound recording] / R.E.M.   
4  2006  09/23/2006 12:41:00 PM                    Document / R.E.M.   

                   author publication_year publisher format_subgroup  \
0  R.E.M. (Musical group)           [1987]   I.R.S.,      Audio Disc   
1  R.E.M. (Musical group)           p1987.   I.R.S.,      Audio Disc   
2  R.E.M. (Musical group)           [1987]   I.R.S.,      Audio Disc   
3  R.E.M. (Musical group)           p1987.   I.R.S.,      Audio Disc   
4  R.E.M. (Musical group)           [1987]   I.R.S.,      Audio Disc   

    description  
0  CD: Adult/YA  
1  CD: Adult/YA  
2  CD: Adult/YA  
3  CD: Adult/YA  
4  CD: Adult/YA  


In [13]:
checkouts.to_csv('./seattle-library-checkout-records/checkouts_total_sample.csv', index=False)

### take a sample without joining on the metadata

In order to avoid any biases with values that are ignored when joining with the inventory list, for the checkout trends I just take a 10% sample without any joining.

In [5]:
dictionary = pd.read_csv('./seattle-library-checkout-records/Integrated_Library_System__ILS__Data_Dictionary.csv')
dictionary.rename(columns={'Code' : 'Collection'}, inplace = True)
print(len(dictionary))
dictionary.head()

555


,Collection,Description,Code Type,Format Group,Format Subgroup,Category Group,Category Subgroup
0,pkbknh,Peak Picks Book,ItemType,NaN,NaN,NaN,NaN
1,acart,Framed Art: Adult/YA,ItemType,Media,Art,NaN,NaN
2,acbk,Book: Adult/YA,ItemType,Print,Book,NaN,NaN
3,accas,Audio Tape: Adult/YA,ItemType,Media,Audio Tape,NaN,NaN
4,accd,CD: Adult/YA,ItemType,Media,Audio Disc,NaN,NaN


In [13]:
checkouts_without_inventory = pd.DataFrame(columns = ['year', 'checkout_time', 'title', 'author', 'publication_year','publisher', 'format_subgroup','description'])

for year in range(2006, 2019):
    print(year)
    df = pd.read_csv('./seattle-library-checkout-records/Checkouts_By_Title_Data_Lens_'+str(year)+'.csv')
    df = pd.merge(df.sample(frac=0.1), dictionary, on ='Collection', how='inner') ##take 10% sample, and join to metadata table
    df.head()
    df.rename(columns = {'CheckoutDateTime': 'checkout_time',\
                       'Format Subgroup':'format_subgroup',\
                       'Description' : 'description'}, inplace = True)

    df['year'] = [year]*len(df)
    
    checkouts_without_inventory = checkouts_without_inventory.append(df[['year','checkout_time','format_subgroup','description']])

2006


//anaconda/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [14]:
checkouts_without_inventory.to_csv('./seattle-library-checkout-records/checkouts_total_sample_withou_inventory_join.csv', index=False)